In [1]:
pip install transformers datasets scikit-learn

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.0 MB 3.9 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/10.0 MB 4.2 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.0 MB 4.7 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.0 MB 4.9 MB/s eta 0:00:02
   ---------------------- ----------------- 5.8/10.0 MB 5.1 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/10.0 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/10.0 MB 5.1 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- -------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.0 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.2 which is incompatible.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from transformers import pipeline
from datasets import load_dataset

# Step 1: Load Dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "all")
queries = dataset["train"]["query"]
contexts = dataset["train"]["retrieved_context"]
labels = dataset["train"]["label"]

# Step 2: Use TF-IDF for Retrieval
vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(contexts)

retrieved_contexts = []
for query in queries:
    query_vector = vectorizer.transform([query])
    similarity_scores = (query_vector @ vectorizer.transform(contexts).T).toarray().flatten()
    best_match_idx = similarity_scores.argmax()
    retrieved_contexts.append(contexts[best_match_idx])

# Step 3: Use RAG for Generated Answers
rag_pipeline = pipeline("question-answering", model="facebook/rag-token-base")
generated_answers = [
    rag_pipeline({"question": query, "context": context})["answer"]
    for query, context in zip(queries, retrieved_contexts)
]

# Step 4: Prepare Data for SVM
X = [" ".join([query, context]) for query, context in zip(queries, retrieved_contexts)]
X_tfidf = vectorizer.fit_transform(X)
y = labels

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Step 6: Train SVM Classifier
svm = SVC(kernel="linear", C=1.0)
svm.fit(X_train, y_train)

# Step 7: Evaluate SVM
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

README.md:   0%|          | 0.00/9.03k [00:00<?, ?B/s]

C:\Users\Lenovo\.conda\envs\pytorch_cuda_gpu\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\datasets--HuggingFaceTB--smoltalk. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00009.parquet:   0%|          | 0.00/223M [00:00<?, ?B/s]

train-00001-of-00009.parquet:   0%|          | 0.00/223M [00:00<?, ?B/s]

train-00002-of-00009.parquet:   0%|          | 0.00/223M [00:00<?, ?B/s]

...

train-00008-of-00009.parquet:   0%|          | 0.00/224M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/54948 [00:00<?, ? examples/s]

KeyError: "Column query not in the dataset. Current columns in the dataset: ['messages', 'source']"

In [3]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "all")
print(dataset["train"].column_names)


['messages', 'source']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from transformers import pipeline
from datasets import load_dataset

# Step 1: Load Dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "all")
queries = dataset["train"]["messages"]  # Extract queries
contexts = dataset["train"]["source"]  # Extract contexts
labels = [0] * len(queries)  # Assign dummy binary labels if labels are not provided

# Step 2: Use TF-IDF for Retrieval
vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(contexts)

retrieved_contexts = []
for query in queries:
    query_vector = vectorizer.transform([query])
    similarity_scores = (query_vector @ vectorizer.transform(contexts).T).toarray().flatten()
    best_match_idx = similarity_scores.argmax()
    retrieved_contexts.append(contexts[best_match_idx])

# Step 3: Use RAG for Generated Answers
rag_pipeline = pipeline("question-answering", model="facebook/rag-token-base")
generated_answers = [
    rag_pipeline({"question": query, "context": context})["answer"]
    for query, context in zip(queries, retrieved_contexts)
]

# Step 4: Prepare Data for SVM
X = [" ".join([query, context]) for query, context in zip(queries, retrieved_contexts)]
X_tfidf = vectorizer.fit_transform(X)

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Step 6: Train SVM Classifier
svm = SVC(kernel="linear", C=1.0)
svm.fit(X_train, y_train)

# Step 7: Evaluate SVM
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))
